# Importing and Inspecting the Data

In [1]:
# import depedencies
import requests as req
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import json
from config import password
from sqlalchemy import create_engine

In [2]:
url_covid = 'https://api.covidtracking.com/v1/states/daily.json'
# url_usa_population = 'https://datausa.io/api/data?drilldowns=Nation&measures=Population&year=latest'
url_state_population = 'https://datausa.io/api/data?drilldowns=State&measures=Population&year=latest'

In [3]:
# retrieving API and storing data in variables
covid_js = req.get(url_covid).json()
# usa_pop_js = req.get(url_usa_population).json()
state_pop_js = req.get(url_state_population).json()
print(json.dumps(state_pop_js['data'], indent=4, sort_keys=True))

[
    {
        "ID State": "04000US30",
        "ID Year": 2018,
        "Population": 1062305,
        "Slug State": "montana",
        "State": "Montana",
        "Year": "2018"
    },
    {
        "ID State": "04000US01",
        "ID Year": 2018,
        "Population": 4887871,
        "Slug State": "alabama",
        "State": "Alabama",
        "Year": "2018"
    },
    {
        "ID State": "04000US04",
        "ID Year": 2018,
        "Population": 7171646,
        "Slug State": "arizona",
        "State": "Arizona",
        "Year": "2018"
    },
    {
        "ID State": "04000US05",
        "ID Year": 2018,
        "Population": 3013825,
        "Slug State": "arkansas",
        "State": "Arkansas",
        "Year": "2018"
    },
    {
        "ID State": "04000US06",
        "ID Year": 2018,
        "Population": 39557045,
        "Slug State": "california",
        "State": "California",
        "Year": "2018"
    },
    {
        "ID State": "04000US08",
        "ID Year": 2

In [4]:
# converting the data to a dataframe for data manipulation and cleaning
covid_all = pd.DataFrame(covid_js)
# usa_pop_df = pd.DataFrame(usa_pop_js['data'])
state_pop_df = pd.DataFrame(state_pop_js['data'])
display(covid_all.head(), state_pop_df.head())

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201020,AK,12432.0,NaN,534093.0,NaN,totalTestsViral,546525.0,38.0,NaN,...,546525,0,0,c0e44ad8d86fcc1e637484d963ee4b2a44cca93c,0,0,0,0,0,
1,20201020,AL,174528.0,21512.0,1112559.0,NaN,totalTestsViral,1265575.0,846.0,19081.0,...,1287087,16,0,1fbf9e94e20368495944d9b840b76b5d3299a657,0,0,0,0,0,
2,20201020,AR,100441.0,6023.0,1137234.0,NaN,totalTestsViral,1231652.0,627.0,6428.0,...,1237675,14,67,696a543f35e5a8e1498d893e4b20a56bae36154a,0,0,0,0,0,
3,20201020,AS,0.0,NaN,1616.0,NaN,totalTestsViral,1616.0,NaN,NaN,...,1616,0,0,3321ebf66fbc98011373f6dcb01d358a9c68c1ac,0,0,0,0,0,
4,20201020,AZ,232937.0,5267.0,1419675.0,NaN,totalTestsPeopleViral,1647345.0,777.0,20719.0,...,1652612,7,71,4be9a6265b0b8e6d09e2cb6bff66e9c7690990f1,0,0,0,0,0,


,ID State,State,ID Year,Year,Population,Slug State
0,04000US30,Montana,2018,2018,1062305,montana
1,04000US01,Alabama,2018,2018,4887871,alabama
2,04000US04,Arizona,2018,2018,7171646,arizona
3,04000US05,Arkansas,2018,2018,3013825,arkansas
4,04000US06,California,2018,2018,39557045,california


In [5]:
##### Emilio's Code Start Here #####

In [6]:
# state_pop_df['Population'].sum()

In [7]:
state_pop_df['State'].nunique()

52

In [8]:
state_pop_df['State'].unique()

array(['Montana', 'Alabama', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Alaska', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'],
      dtype=object)

In [9]:
# inspecting data
display(covid_all.shape, state_pop_df.shape)

(12933, 55)

(52, 6)

In [10]:
# inspecting data
covid_all.describe()

,date,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,total,totalTestResultsIncrease,posNeg,deathIncrease,hospitalizedIncrease,commercialScore,negativeRegularScore,negativeScore,positiveScore,score
count,1.293300e+04,12823.000000,3959.000000,1.268500e+04,1449.000000,1.290300e+04,10023.000000,7442.000000,5782.000000,2157.00000,...,1.293300e+04,12933.000000,1.293300e+04,12933.000000,12933.000000,12933.0,12933.0,12933.0,12933.0,12933.0
mean,2.020065e+07,59252.125322,3300.621116,6.888098e+05,1309.364389,7.700717e+05,850.223885,7246.263908,295.333276,1123.10663,...,7.344962e+05,9815.763551,7.343495e+05,16.444599,33.941313,0.0,0.0,0.0,0.0,0.0
std,2.203379e+02,114199.840170,4389.123980,1.458547e+06,5221.300997,1.567902e+06,1593.419594,14730.659193,531.101970,1074.87762,...,1.549193e+06,18429.967211,1.549146e+06,45.024185,224.593712,0.0,0.0,0.0,0.0,0.0
min,2.020012e+07,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,6.00000,...,0.000000e+00,-336892.000000,0.000000e+00,-213.000000,-4124.000000,0.0,0.0,0.0,0.0,0.0
25%,2.020043e+07,1846.500000,468.000000,3.449400e+04,21.000000,3.337800e+04,106.000000,581.250000,55.000000,288.00000,...,3.239500e+04,605.000000,3.198500e+04,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
50%,2.020063e+07,15839.000000,1807.000000,2.053530e+05,176.000000,2.378030e+05,379.000000,2460.000000,134.000000,838.00000,...,2.158750e+05,3543.000000,2.158750e+05,4.000000,0.000000,0.0,0.0,0.0,0.0,0.0
75%,2.020082e+07,66558.000000,4327.500000,6.985540e+05,583.000000,7.917080e+05,878.000000,7600.250000,282.000000,1551.00000,...,7.495020e+05,11020.000000,7.495020e+05,14.000000,24.000000,0.0,0.0,0.0,0.0,0.0
max,2.020102e+07,874077.000000,29058.000000,1.631499e+07,64400.000000,1.718907e+07,18825.000000,89995.000000,5225.000000,5701.00000,...,1.718907e+07,187926.000000,1.718907e+07,951.000000,16373.000000,0.0,0.0,0.0,0.0,0.0


In [11]:
# # inspecting data: fields (also by reviewing API documentation)
covid_all.columns

Index(['date', 'state', 'positive', 'probableCases', 'negative', 'pending',
       'totalTestResultsSource', 'totalTestResults', 'hospitalizedCurrently',
       'hospitalizedCumulative', 'inIcuCurrently', 'inIcuCumulative',
       'onVentilatorCurrently', 'onVentilatorCumulative', 'recovered',
       'dataQualityGrade', 'lastUpdateEt', 'dateModified', 'checkTimeEt',
       'death', 'hospitalized', 'dateChecked', 'totalTestsViral',
       'positiveTestsViral', 'negativeTestsViral', 'positiveCasesViral',
       'deathConfirmed', 'deathProbable', 'totalTestEncountersViral',
       'totalTestsPeopleViral', 'totalTestsAntibody', 'positiveTestsAntibody',
       'negativeTestsAntibody', 'totalTestsPeopleAntibody',
       'positiveTestsPeopleAntibody', 'negativeTestsPeopleAntibody',
       'totalTestsPeopleAntigen', 'positiveTestsPeopleAntigen',
       'totalTestsAntigen', 'positiveTestsAntigen', 'fips', 'positiveIncrease',
       'negativeIncrease', 'total', 'totalTestResultsIncrease', 'posNe

In [12]:
# # inspecting data: data types
covid_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12933 entries, 0 to 12932
Data columns (total 55 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   date                         12933 non-null  int64  
 1   state                        12933 non-null  object 
 2   positive                     12823 non-null  float64
 3   probableCases                3959 non-null   float64
 4   negative                     12685 non-null  float64
 5   pending                      1449 non-null   float64
 6   totalTestResultsSource       12933 non-null  object 
 7   totalTestResults             12903 non-null  float64
 8   hospitalizedCurrently        10023 non-null  float64
 9   hospitalizedCumulative       7442 non-null   float64
 10  inIcuCurrently               5782 non-null   float64
 11  inIcuCumulative              2157 non-null   float64
 12  onVentilatorCurrently        4782 non-null   float64
 13  onVentilatorCumu

# Data Cleaning
## covid_df
#### Removing the following columns as they don't return required data, as well as deprecated fields reported in the documentation:

In [13]:
# list of fields: all, not-required, and deprecated fields
all_fields = ['date', 'state', 'positive', 'probableCases', 'negative', 'pending',
       'totalTestResults', 'hospitalizedCurrently', 'hospitalizedCumulative',
       'inIcuCurrently', 'inIcuCumulative', 'onVentilatorCurrently',
       'onVentilatorCumulative', 'recovered', 'dataQualityGrade',
       'lastUpdateEt', 'dateModified', 'checkTimeEt', 'death', 'hospitalized',
       'dateChecked', 'totalTestsViral', 'positiveTestsViral',
       'negativeTestsViral', 'positiveCasesViral', 'deathConfirmed',
       'deathProbable', 'totalTestEncountersViral', 'totalTestsPeopleViral',
       'totalTestsAntibody', 'positiveTestsAntibody', 'negativeTestsAntibody',
       'totalTestsPeopleAntibody', 'positiveTestsPeopleAntibody',
       'negativeTestsPeopleAntibody', 'totalTestsPeopleAntigen',
       'positiveTestsPeopleAntigen', 'totalTestsAntigen',
       'positiveTestsAntigen', 'fips', 'positiveIncrease', 'negativeIncrease',
       'total', 'totalTestResultsSource', 'totalTestResultsIncrease', 'posNeg',
       'deathIncrease', 'hospitalizedIncrease', 'hash', 'commercialScore',
       'negativeRegularScore', 'negativeScore', 'positiveScore', 'score',
       'grade']
deprecated_fields = ['checkTimeEt', 'commercialScore', 'dateChecked', 'dateModified', 
                     'grade', 'hash', 'hospitalized', 'negativeIncrease', 
                     'negativeRegularScore', 'negativeScore', 'posNeg', 'positiveScore', 
                     'score', 'total', 'totalTestResultsSource']
non_required_fields = ['deathConfirmed', 'deathProbable', 'lastUpdateEt', 
                       'totalTestsViral', 'positiveTestsViral', 'negativeTestsViral', 
                       'positiveCasesViral', 'probableCases', 'negative', 'recovered',
                       'pending', 'totalTestEncountersViral', 'totalTestsPeopleViral', 
                       'totalTestsAntibody', 'positiveTestsAntibody', 'negativeTestsAntibody',
                       'totalTestsPeopleAntibody', 'positiveTestsPeopleAntibody',
                       'negativeTestsPeopleAntibody', 'totalTestsPeopleAntigen', 
                       'positiveTestsPeopleAntigen','totalTestsAntigen', 'positiveTestsAntigen', 
                       'totalTestResultsIncrease', 'totalTestResults']

In [14]:
# removing deprecated and non-required fields to obtain the filtered list
filtered_fields = []
for i in all_fields:
    if i not in deprecated_fields and i not in non_required_fields:
        filtered_fields.append(i)
filtered_fields

['date',
 'state',
 'positive',
 'hospitalizedCurrently',
 'hospitalizedCumulative',
 'inIcuCurrently',
 'inIcuCumulative',
 'onVentilatorCurrently',
 'onVentilatorCumulative',
 'dataQualityGrade',
 'death',
 'fips',
 'positiveIncrease',
 'deathIncrease',
 'hospitalizedIncrease']

In [15]:
# new DF with the filtered fields
covid_df = covid_all[filtered_fields]
covid_df.head() # still think 23 columns is a lot to deal with for the prohect

,date,state,positive,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,dataQualityGrade,death,fips,positiveIncrease,deathIncrease,hospitalizedIncrease
0,20201020,AK,12432.0,38.0,NaN,NaN,NaN,10.0,NaN,A,67.0,02,212,0,0
1,20201020,AL,174528.0,846.0,19081.0,NaN,1954.0,NaN,1114.0,A,2805.0,01,1043,16,0
2,20201020,AR,100441.0,627.0,6428.0,257.0,NaN,96.0,782.0,A+,1728.0,05,844,14,67
3,20201020,AS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,D,0.0,60,0,0,0
4,20201020,AZ,232937.0,777.0,20719.0,170.0,NaN,91.0,NaN,A+,5837.0,04,1040,7,71


In [16]:
# converting date-string format to date
covid_df['date'] = pd.to_datetime(covid_df['date'], format='%Y%m%d')
covid_df.head()

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date,state,positive,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,dataQualityGrade,death,fips,positiveIncrease,deathIncrease,hospitalizedIncrease
0,2020-10-20,AK,12432.0,38.0,NaN,NaN,NaN,10.0,NaN,A,67.0,02,212,0,0
1,2020-10-20,AL,174528.0,846.0,19081.0,NaN,1954.0,NaN,1114.0,A,2805.0,01,1043,16,0
2,2020-10-20,AR,100441.0,627.0,6428.0,257.0,NaN,96.0,782.0,A+,1728.0,05,844,14,67
3,2020-10-20,AS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,D,0.0,60,0,0,0
4,2020-10-20,AZ,232937.0,777.0,20719.0,170.0,NaN,91.0,NaN,A+,5837.0,04,1040,7,71


### More Inspection

In [17]:
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12933 entries, 0 to 12932
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    12933 non-null  datetime64[ns]
 1   state                   12933 non-null  object        
 2   positive                12823 non-null  float64       
 3   hospitalizedCurrently   10023 non-null  float64       
 4   hospitalizedCumulative  7442 non-null   float64       
 5   inIcuCurrently          5782 non-null   float64       
 6   inIcuCumulative         2157 non-null   float64       
 7   onVentilatorCurrently   4782 non-null   float64       
 8   onVentilatorCumulative  750 non-null    float64       
 9   dataQualityGrade        11740 non-null  object        
 10  death                   12173 non-null  float64       
 11  fips                    12933 non-null  object        
 12  positiveIncrease        12933 non-null  int64 

In [18]:
# validating results by state and inspecting dates
covid_df.groupby(['state']).agg({'positiveIncrease': 'sum', 'deathIncrease': 'sum', 'date': 'count', 'date': ['min', 'max']})
# https://covid.cdc.gov/covid-data-tracker/#cases_casesinlast7days

positiveIncrease deathIncrease       date           
                   sum           sum        min        max
state                                                     
AK               12432            67 2020-03-06 2020-10-20
AL              174528          2805 2020-03-07 2020-10-20
AR              100441          1728 2020-03-06 2020-10-20
AS                   0             0 2020-03-16 2020-10-20
AZ              232935          5837 2020-03-04 2020-10-20
CA              874024         16992 2020-03-04 2020-10-20
CO               86372          2051 2020-03-05 2020-10-20
CT               64455          4559 2020-03-07 2020-10-20
DC               16445           642 2020-03-05 2020-10-20
DE               23325           668 2020-03-06 2020-10-20
FL              760387         16308 2020-03-04 2020-10-20
GA              342436          7674 2020-03-04 2020-10-20
GU                3883            67 2020-03-16 2020-10-20
HI               14262           187 2020-03-04 2020-10-20
IA              103011          1555 2020-03-06 2020-10-20
ID               53790           531 2020-03-07 2020-10-20
IL              354458          9537 2020-03-04 2020-10-20
IN              150664          4008 2020-02-27 2020-10-20
KS               72968           872 2020-03-06 2020-10-20
KY               89544          1342 2020-03-06 2020-10-20
LA              179769          5772 2020-03-07 2020-10-20
MA              145464          9758 2020-01-22 2020-10-20
MD              136744          4050 2020-03-05 2020-10-20
ME                5989           146 2020-03-07 2020-10-20
MI              165980          7383 2020-03-01 2020-10-20
MN              125530          2246 2020-03-06 2020-10-20
MO              159625          2615 2020-03-07 2020-10-20
MP                  88             2 2020-03-16 2020-10-20
MS              111322          3202 2020-03-07 2020-10-20
MT               24093           252 2020-03-07 2020-10-20
NC              248749          3992 2020-03-04 2020-10-20
ND               33666           315 2020-03-07 2020-10-20
NE               58817           554 2020-03-05 2020-10-20
NH                9826           468 2020-03-04 2020-10-20
NJ              222193         16227 2020-02-10 2020-10-20
NM               37302           935 2020-03-06 2020-10-20
NV               91498          1727 2020-03-05 2020-10-20
NY              486474         25672 2020-03-04 2020-10-20
OH              185639          5083 2020-03-05 2020-10-20
OK              109547          1191 2020-03-07 2020-10-20
OR               40133           633 2020-03-04 2020-10-20
PA              184870          8533 2020-03-06 2020-10-20
PR               58638           769 2020-03-16 2020-10-20
RI               28647          1164 2020-03-01 2020-10-20
SC              165493          3696 2020-03-04 2020-10-20
SD               34457           330 2020-03-07 2020-10-20
TN              233568          2952 2020-03-05 2020-10-20
TX              833556         17087 2020-03-04 2020-10-20
UT               96642           551 2020-03-07 2020-10-20
VA              167754          3485 2020-02-27 2020-10-20
VI                1334            21 2020-03-16 2020-10-20
VT                1956            58 2020-03-06 2020-10-20
WA               98661          2258 2020-01-22 2020-10-20
WI              187827          1652 2020-03-04 2020-10-20
WV               20519           408 2020-03-06 2020-10-20
WY                9526            61 2020-03-07 2020-10-20

In [19]:
# notice the count of dates is uneven, which is reflected by the start dates variation


In [20]:
# inspecting viz
covid_viz = covid_df.set_index(['state', 'date']).sort_index()
idx = pd.IndexSlice
select_viz = covid_viz.loc[idx[['MN'], :], :]
select_viz.positiveIncrease

state  date      
MN     2020-03-06       0
       2020-03-07       1
       2020-03-08       0
       2020-03-09       3
       2020-03-10       6
                     ... 
       2020-10-16    2290
       2020-10-17    1694
       2020-10-18    1722
       2020-10-19    1627
       2020-10-20    1092
Name: positiveIncrease, Length: 229, dtype: int64

# note: we could slice dates based on variables defined by a slicer on the dashboard

# Data Cleaning
## state_pop_df

In [21]:
# inspecting columns
state_pop_df.columns

Index(['ID State', 'State', 'ID Year', 'Year', 'Population', 'Slug State'], dtype='object')

In [22]:
state_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ID State    52 non-null     object
 1   State       52 non-null     object
 2   ID Year     52 non-null     int64 
 3   Year        52 non-null     object
 4   Population  52 non-null     int64 
 5   Slug State  52 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.6+ KB


In [23]:
# defining fips field from ID state. This new column will be used for merging
state_pop_df['fips'] = state_pop_df['ID State'].str[-2:]

In [24]:
state_df = state_pop_df[['State', 'fips', 'Population']]
state_df.head()

,State,fips,Population
0,Montana,30,1062305
1,Alabama,01,4887871
2,Arizona,04,7171646
3,Arkansas,05,3013825
4,California,06,39557045


# Merging Datasets

In [25]:
covid_df['fips'].unique()

array(['02', '01', '05', '60', '04', '06', '08', '09', '11', '10', '12',
       '13', '66', '15', '19', '16', '17', '18', '20', '21', '22', '25',
       '24', '23', '26', '27', '29', '69', '28', '30', '37', '38', '31',
       '33', '34', '35', '32', '36', '39', '40', '41', '42', '72', '44',
       '45', '46', '47', '48', '49', '51', '78', '50', '53', '55', '54',
       '56'], dtype=object)

In [26]:
covid_wpop = covid_df.merge(state_df, how='inner', on='fips')
covid_wpop.head()

,date,state,positive,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,dataQualityGrade,death,fips,positiveIncrease,deathIncrease,hospitalizedIncrease,State,Population
0,2020-10-20,AK,12432.0,38.0,NaN,NaN,NaN,10.0,NaN,A,67.0,02,212,0,0,Alaska,737438
1,2020-10-19,AK,12220.0,65.0,NaN,NaN,NaN,8.0,NaN,A,67.0,02,204,0,0,Alaska,737438
2,2020-10-18,AK,12016.0,73.0,NaN,NaN,NaN,11.0,NaN,A,67.0,02,224,0,0,Alaska,737438
3,2020-10-17,AK,11792.0,68.0,NaN,NaN,NaN,9.0,NaN,A,67.0,02,216,1,0,Alaska,737438
4,2020-10-16,AK,11576.0,59.0,NaN,NaN,NaN,8.0,NaN,A,66.0,02,228,1,0,Alaska,737438


In [27]:
covid_wpop[covid_wpop.Population == 0]

,date,state,positive,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,dataQualityGrade,death,fips,positiveIncrease,deathIncrease,hospitalizedIncrease,State,Population


In [28]:
covid_wpop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12057 entries, 0 to 12056
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    12057 non-null  datetime64[ns]
 1   state                   12057 non-null  object        
 2   positive                11962 non-null  float64       
 3   hospitalizedCurrently   9857 non-null   float64       
 4   hospitalizedCumulative  7338 non-null   float64       
 5   inIcuCurrently          5668 non-null   float64       
 6   inIcuCumulative         2157 non-null   float64       
 7   onVentilatorCurrently   4756 non-null   float64       
 8   onVentilatorCumulative  750 non-null    float64       
 9   dataQualityGrade        11024 non-null  object        
 10  death                   11346 non-null  float64       
 11  fips                    12057 non-null  object        
 12  positiveIncrease        12057 non-null  int64 

In [29]:
display(covid_wpop['state'].nunique(), covid_df['state'].nunique())

52

56

In [30]:
display(covid_wpop['state'].unique(), covid_df['state'].unique())

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
       'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

array(['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL',
       'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA',
       'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE',
       'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI',
       'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV',
       'WY'], dtype=object)

In [31]:
# checking for omitted states: the result are US territories.
omitted_states = []
for i in covid_df['state'].unique():
    if i not in covid_wpop['state'].unique():
        omitted_states.append(i)
omitted_states

['AS', 'GU', 'MP', 'VI']

### Seven-day rolling average of new cases, by number of days since X average daily cases first recorded

In [32]:
covid_indexed = covid_wpop.set_index(['state']).sort_index().sort_values(['state', 'date'])

In [33]:
covid_rollavg = covid_indexed[['date', 'positive', 'positiveIncrease', 'death', 'deathIncrease']]

In [34]:
covid_rollavg['positiveRollingAvg'] = covid_rollavg.groupby(level=0)['positiveIncrease'].rolling(window=7).mean().values
covid_rollavg

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,date,positive,positiveIncrease,death,deathIncrease,positiveRollingAvg
state,,,,,,
AK,2020-03-06,0.0,0,0.0,0,NaN
AK,2020-03-07,0.0,0,0.0,0,NaN
AK,2020-03-08,0.0,0,0.0,0,NaN
AK,2020-03-09,0.0,0,0.0,0,NaN
AK,2020-03-10,0.0,0,0.0,0,NaN
...,...,...,...,...,...,...
WY,2020-10-16,8665.0,290,57.0,0,190.000000
WY,2020-10-17,8816.0,151,57.0,0,194.428571
WY,2020-10-18,9025.0,209,57.0,0,202.000000


In [35]:
covid_rollavg['deathRollingAvg'] = covid_rollavg.groupby(level=0)['deathIncrease'].rolling(window=7).mean().values

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
# new cases: looping to get the date >= threshold for each state
state_list = covid_rollavg.index.get_level_values('state').unique()
new_threshold = 10
dates_new = []
for i in state_list:
    new_df = covid_rollavg.loc[i]
    new_df = new_df.loc[new_df['positiveIncrease'] >= new_threshold]
    date = new_df.iloc[0].date
    dates_new.append(date)
dates_new

[Timestamp('2020-03-24 00:00:00'),
 Timestamp('2020-03-16 00:00:00'),
 Timestamp('2020-03-18 00:00:00'),
 Timestamp('2020-03-19 00:00:00'),
 Timestamp('2020-03-08 00:00:00'),
 Timestamp('2020-03-11 00:00:00'),
 Timestamp('2020-03-17 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-03-22 00:00:00'),
 Timestamp('2020-03-13 00:00:00'),
 Timestamp('2020-03-13 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-03-21 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-03-10 00:00:00'),
 Timestamp('2020-03-19 00:00:00'),
 Timestamp('2020-03-19 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-03-13 00:00:00'),
 Timestamp('2020-03-13 00:00:00'),
 Timestamp('2020-03-17 00:00:00'),
 Timestamp('2020-03-16 00:00:00'),
 Timestamp('2020-03-02 00:00:00'),
 Timestamp('2020-03-11 00:00:00'),
 Timestamp('2020-03-19 00:00:00'),
 Timestamp('2020-03-18 00:00:00'),
 Timestamp('2020-03-22 00:00:00'),
 Timestamp('2020-03-18 00:00:00'),
 Timestamp('2020-03-

In [37]:
new_dict = dict(zip(state_list, dates_new))
new_dict

{'AK': Timestamp('2020-03-24 00:00:00'),
 'AL': Timestamp('2020-03-16 00:00:00'),
 'AR': Timestamp('2020-03-18 00:00:00'),
 'AZ': Timestamp('2020-03-19 00:00:00'),
 'CA': Timestamp('2020-03-08 00:00:00'),
 'CO': Timestamp('2020-03-11 00:00:00'),
 'CT': Timestamp('2020-03-17 00:00:00'),
 'DC': Timestamp('2020-03-20 00:00:00'),
 'DE': Timestamp('2020-03-22 00:00:00'),
 'FL': Timestamp('2020-03-13 00:00:00'),
 'GA': Timestamp('2020-03-13 00:00:00'),
 'HI': Timestamp('2020-03-20 00:00:00'),
 'IA': Timestamp('2020-03-21 00:00:00'),
 'ID': Timestamp('2020-03-20 00:00:00'),
 'IL': Timestamp('2020-03-10 00:00:00'),
 'IN': Timestamp('2020-03-19 00:00:00'),
 'KS': Timestamp('2020-03-19 00:00:00'),
 'KY': Timestamp('2020-03-20 00:00:00'),
 'LA': Timestamp('2020-03-13 00:00:00'),
 'MA': Timestamp('2020-03-13 00:00:00'),
 'MD': Timestamp('2020-03-17 00:00:00'),
 'ME': Timestamp('2020-03-16 00:00:00'),
 'MI': Timestamp('2020-03-02 00:00:00'),
 'MN': Timestamp('2020-03-11 00:00:00'),
 'MO': Timestamp

In [38]:
covid_rollavg.head()

,date,positive,positiveIncrease,death,deathIncrease,positiveRollingAvg,deathRollingAvg
state,,,,,,,
AK,2020-03-06,0.0,0,0.0,0,NaN,NaN
AK,2020-03-07,0.0,0,0.0,0,NaN,NaN
AK,2020-03-08,0.0,0,0.0,0,NaN,NaN
AK,2020-03-09,0.0,0,0.0,0,NaN,NaN
AK,2020-03-10,0.0,0,0.0,0,NaN,NaN


In [39]:
covid_rollavg.iloc[0].date

Timestamp('2020-03-06 00:00:00')

In [40]:
# deaths: looping to get the date >= threshold for each state
death_threshold = 10
dates_death = []
for i in state_list:
    death_df = covid_rollavg.loc[i]
    death_df = death_df.loc[death_df['death'] >= death_threshold]
    date_d = death_df.iloc[0].date
    dates_death.append(date_d)
dates_death

[Timestamp('2020-05-06 00:00:00'),
 Timestamp('2020-03-31 00:00:00'),
 Timestamp('2020-04-01 00:00:00'),
 Timestamp('2020-03-27 00:00:00'),
 Timestamp('2020-03-17 00:00:00'),
 Timestamp('2020-03-25 00:00:00'),
 Timestamp('2020-03-23 00:00:00'),
 Timestamp('2020-04-01 00:00:00'),
 Timestamp('2020-03-29 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-03-19 00:00:00'),
 Timestamp('2020-04-20 00:00:00'),
 Timestamp('2020-04-02 00:00:00'),
 Timestamp('2020-04-04 00:00:00'),
 Timestamp('2020-03-23 00:00:00'),
 Timestamp('2020-03-24 00:00:00'),
 Timestamp('2020-04-01 00:00:00'),
 Timestamp('2020-03-31 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-03-22 00:00:00'),
 Timestamp('2020-03-27 00:00:00'),
 Timestamp('2020-04-04 00:00:00'),
 Timestamp('2020-03-19 00:00:00'),
 Timestamp('2020-03-30 00:00:00'),
 Timestamp('2020-03-28 00:00:00'),
 Timestamp('2020-03-28 00:00:00'),
 Timestamp('2020-04-18 00:00:00'),
 Timestamp('2020-04-01 00:00:00'),
 Timestamp('2020-04-

In [41]:
death_dict = dict(zip(state_list, dates_death))
death_dict

{'AK': Timestamp('2020-05-06 00:00:00'),
 'AL': Timestamp('2020-03-31 00:00:00'),
 'AR': Timestamp('2020-04-01 00:00:00'),
 'AZ': Timestamp('2020-03-27 00:00:00'),
 'CA': Timestamp('2020-03-17 00:00:00'),
 'CO': Timestamp('2020-03-25 00:00:00'),
 'CT': Timestamp('2020-03-23 00:00:00'),
 'DC': Timestamp('2020-04-01 00:00:00'),
 'DE': Timestamp('2020-03-29 00:00:00'),
 'FL': Timestamp('2020-03-20 00:00:00'),
 'GA': Timestamp('2020-03-19 00:00:00'),
 'HI': Timestamp('2020-04-20 00:00:00'),
 'IA': Timestamp('2020-04-02 00:00:00'),
 'ID': Timestamp('2020-04-04 00:00:00'),
 'IL': Timestamp('2020-03-23 00:00:00'),
 'IN': Timestamp('2020-03-24 00:00:00'),
 'KS': Timestamp('2020-04-01 00:00:00'),
 'KY': Timestamp('2020-03-31 00:00:00'),
 'LA': Timestamp('2020-03-20 00:00:00'),
 'MA': Timestamp('2020-03-22 00:00:00'),
 'MD': Timestamp('2020-03-27 00:00:00'),
 'ME': Timestamp('2020-04-04 00:00:00'),
 'MI': Timestamp('2020-03-19 00:00:00'),
 'MN': Timestamp('2020-03-30 00:00:00'),
 'MO': Timestamp

In [42]:
covid_rollavg.reset_index(inplace=True)

In [43]:
covid_rollavg['new_datelapse'] = ''

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
covid_rollavg['death_datelapse'] = ''

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [45]:
# populating new_datelapse with the dete substraction for number of days since reaching the threshold
for x in covid_rollavg.index:
    covid_rollavg.iloc[x, 8] = (covid_rollavg.iloc[x, 1] - new_dict[covid_rollavg.iloc[x, 0]]).days

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [46]:
# populating death_datelapse with the dete substraction for number of days since reaching the threshold
for x in covid_rollavg.index:
    covid_rollavg.iloc[x, 9] = (covid_rollavg.iloc[x, 1] - death_dict[covid_rollavg.iloc[x, 0]]).days

In [47]:
covid_rollavg['new_datelapse'] = pd.to_numeric(covid_rollavg['new_datelapse'], errors='coerce')


/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [48]:
covid_rollavg['death_datelapse'] = pd.to_numeric(covid_rollavg['death_datelapse'], errors='coerce')

/opt/anaconda3/envs/NewPythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
covid_rollavg = covid_rollavg.rename(columns={'positiveIncrease':'positive_increase', 'deathIncrease':'death_increase', 'positiveRollingAvg':'positive_rolling_avg', 'deathRollingAvg':'death_rolling_avg'})
covid_rollavg



,state,date,positive,positive_increase,death,death_increase,positive_rolling_avg,death_rolling_avg,new_datelapse,death_datelapse
0,AK,2020-03-06,0.0,0,0.0,0,NaN,NaN,-18,-61
1,AK,2020-03-07,0.0,0,0.0,0,NaN,NaN,-17,-60
2,AK,2020-03-08,0.0,0,0.0,0,NaN,NaN,-16,-59
3,AK,2020-03-09,0.0,0,0.0,0,NaN,NaN,-15,-58
4,AK,2020-03-10,0.0,0,0.0,0,NaN,NaN,-14,-57
...,...,...,...,...,...,...,...,...,...,...
12052,WY,2020-10-16,8665.0,290,57.0,0,190.000000,0.428571,205,150
12053,WY,2020-10-17,8816.0,151,57.0,0,194.428571,0.428571,206,151
12054,WY,2020-10-18,9025.0,209,57.0,0,202.000000,0.428571,207,152
12055,WY,2020-10-19,9311.0,286,57.0,0,215.571429,0.428571,208,153


In [50]:
## IMPORT INTO POSTGRESQL

rds_connection_string = "postgres:" + password + "@localhost:5432/covid19_db"

In [51]:
engine = create_engine(f'postgresql://{rds_connection_string}')

In [52]:
covid_rollavg.to_sql(name='covid_rolling', con=engine, if_exists='replace', index=True)


In [53]:
covid_rolling = pd.read_sql_query('select * from covid_rolling', con=engine).head(15)
covid_rolling

,index,state,date,positive,positive_increase,death,death_increase,positive_rolling_avg,death_rolling_avg,new_datelapse,death_datelapse
0,0,AK,2020-03-06,0.0,0,0.0,0,NaN,NaN,-18,-61
1,1,AK,2020-03-07,0.0,0,0.0,0,NaN,NaN,-17,-60
2,2,AK,2020-03-08,0.0,0,0.0,0,NaN,NaN,-16,-59
3,3,AK,2020-03-09,0.0,0,0.0,0,NaN,NaN,-15,-58
4,4,AK,2020-03-10,0.0,0,0.0,0,NaN,NaN,-14,-57
5,5,AK,2020-03-11,0.0,0,0.0,0,NaN,NaN,-13,-56
6,6,AK,2020-03-12,0.0,0,0.0,0,0.000000,0.0,-12,-55
7,7,AK,2020-03-13,1.0,1,0.0,0,0.142857,0.0,-11,-54
8,8,AK,2020-03-14,1.0,0,0.0,0,0.142857,0.0,-10,-53
9,9,AK,2020-03-15,1.0,0,0.0,0,0.142857,0.0,-9,-52


---

## Map Viz DF

In [54]:
# call the latest available date for each state
# latest_date = covid_wpop.date == covid_wpop.date.max() this works, but what if one of the states is not updated to the latest date?
# need an alternative solution to call the last date, TBD in class
latest_date = covid_wpop.date == covid_wpop.date.max()
map_df = covid_wpop[latest_date]
map_df

,date,state,positive,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,dataQualityGrade,death,fips,positiveIncrease,deathIncrease,hospitalizedIncrease,State,Population
0,2020-10-20,AK,12432.0,38.0,NaN,NaN,NaN,10.0,NaN,A,67.0,02,212,0,0,Alaska,737438
229,2020-10-20,AL,174528.0,846.0,19081.0,NaN,1954.0,NaN,1114.0,A,2805.0,01,1043,16,0,Alabama,4887871
457,2020-10-20,AR,100441.0,627.0,6428.0,257.0,NaN,96.0,782.0,A+,1728.0,05,844,14,67,Arkansas,3013825
686,2020-10-20,AZ,232937.0,777.0,20719.0,170.0,NaN,91.0,NaN,A+,5837.0,04,1040,7,71,Arizona,7171646
917,2020-10-20,CA,874077.0,3104.0,NaN,781.0,NaN,NaN,NaN,B,16992.0,06,3286,22,0,California,39557045
1148,2020-10-20,CO,86374.0,465.0,8248.0,NaN,NaN,NaN,NaN,A,2051.0,08,1072,9,21,Colorado,5695564
1378,2020-10-20,CT,64455.0,217.0,12043.0,NaN,NaN,NaN,NaN,B,4559.0,09,434,5,0,Connecticut,3572665
1606,2020-10-20,DC,16445.0,85.0,NaN,20.0,NaN,9.0,NaN,A+,642.0,11,50,1,0,District of Columbia,702455
1836,2020-10-20,DE,23325.0,107.0,NaN,25.0,NaN,NaN,NaN,A+,668.0,10,129,2,0,Delaware,967171
2065,2020-10-20,FL,760389.0,2094.0,47895.0,NaN,NaN,NaN,NaN,A,16308.0,12,3662,86,231,Florida,21299325


In [55]:
map_df.shape

(52, 17)

In [56]:
map_df.columns

Index(['date', 'state', 'positive', 'hospitalizedCurrently',
       'hospitalizedCumulative', 'inIcuCurrently', 'inIcuCumulative',
       'onVentilatorCurrently', 'onVentilatorCumulative', 'dataQualityGrade',
       'death', 'fips', 'positiveIncrease', 'deathIncrease',
       'hospitalizedIncrease', 'State', 'Population'],
      dtype='object')

In [57]:
# reorganizing columns
map_df = map_df[['date', 'State', 'state', 'Population', 
                 'positive', 'positiveIncrease', 'death', 'deathIncrease', 
                 'hospitalizedCurrently', 'hospitalizedIncrease', 'hospitalizedCumulative', 
                 'inIcuCurrently', 'inIcuCumulative', 
                 'onVentilatorCurrently', 'onVentilatorCumulative', 
                 'dataQualityGrade']]
map_df.head()

,date,State,state,Population,positive,positiveIncrease,death,deathIncrease,hospitalizedCurrently,hospitalizedIncrease,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,dataQualityGrade
0,2020-10-20,Alaska,AK,737438,12432.0,212,67.0,0,38.0,0,NaN,NaN,NaN,10.0,NaN,A
229,2020-10-20,Alabama,AL,4887871,174528.0,1043,2805.0,16,846.0,0,19081.0,NaN,1954.0,NaN,1114.0,A
457,2020-10-20,Arkansas,AR,3013825,100441.0,844,1728.0,14,627.0,67,6428.0,257.0,NaN,96.0,782.0,A+
686,2020-10-20,Arizona,AZ,7171646,232937.0,1040,5837.0,7,777.0,71,20719.0,170.0,NaN,91.0,NaN,A+
917,2020-10-20,California,CA,39557045,874077.0,3286,16992.0,22,3104.0,0,NaN,781.0,NaN,NaN,NaN,B


In [58]:
map_df['positive_per_100K'] = (map_df['positive'] / (map_df['Population'] / 100000)).round(1)
map_df['death_per_100K'] = (map_df['death'] / (map_df['Population'] / 100000)).round(1)
map_df.head()

,date,State,state,Population,positive,positiveIncrease,death,deathIncrease,hospitalizedCurrently,hospitalizedIncrease,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,onVentilatorCumulative,dataQualityGrade,positive_per_100K,death_per_100K
0,2020-10-20,Alaska,AK,737438,12432.0,212,67.0,0,38.0,0,NaN,NaN,NaN,10.0,NaN,A,1685.8,9.1
229,2020-10-20,Alabama,AL,4887871,174528.0,1043,2805.0,16,846.0,0,19081.0,NaN,1954.0,NaN,1114.0,A,3570.6,57.4
457,2020-10-20,Arkansas,AR,3013825,100441.0,844,1728.0,14,627.0,67,6428.0,257.0,NaN,96.0,782.0,A+,3332.7,57.3
686,2020-10-20,Arizona,AZ,7171646,232937.0,1040,5837.0,7,777.0,71,20719.0,170.0,NaN,91.0,NaN,A+,3248.0,81.4
917,2020-10-20,California,CA,39557045,874077.0,3286,16992.0,22,3104.0,0,NaN,781.0,NaN,NaN,NaN,B,2209.7,43.0


In [59]:
# CASES AND DEATH LAST 7 DAYS?

In [60]:
##### Emilio's Code Ends Here #####

In [61]:
#### Allan's Code Starts ####

In [62]:
#### Allan's Code Ends #### 

In [63]:
#### Matt's Code Starts ####

In [64]:
map_df = map_df.rename(columns={'state':'state_abbr', 'State':'state', 'Population':'population', 'positiveIncrease':'positive_increase', 'deathIncrease':'death_increase', 'hospitalizedCurrently':'hospitalized_currently', 'hospitalizedIncrease':'hospitalized_increase', 'hospitalizedCumulative':'hospitalized_cumulative', 'inIcuCurrently':'in_icu_currently', 'inIcuCumulative':'in_icu_cumulative','onVentilatorCurrently':'on_ventilator_currently', 'onVentilatorCumulative':'on_ventilator_cumulative','dataQualityGrade':'data_quality_grade', 'positive_per_100K':'positive_per_100k', 'death_per_100K':'death_per_100k'})

In [65]:
map_df = map_df.fillna(0)
map_df.dtypes

date                        datetime64[ns]
state                               object
state_abbr                          object
population                           int64
positive                           float64
positive_increase                    int64
death                              float64
death_increase                       int64
hospitalized_currently             float64
hospitalized_increase                int64
hospitalized_cumulative            float64
in_icu_currently                   float64
in_icu_cumulative                  float64
on_ventilator_currently            float64
on_ventilator_cumulative           float64
data_quality_grade                  object
positive_per_100k                  float64
death_per_100k                     float64
dtype: object

In [66]:
rds_connection_string = "postgres:" + password + "@localhost:5432/covid19_db"

In [67]:
engine = create_engine(f'postgresql://{rds_connection_string}')

In [68]:
map_df.to_sql(name='covid_data', con=engine, if_exists='replace', index=False)


In [69]:
mapping_df = pd.read_sql_query('select * from covid_data', con=engine).head(15)


In [70]:
### GET US TOTAL HISTORICAL NUMBERS ###

us_hist_covid = 'https://api.covidtracking.com/v1/us/daily.json'
    
covid_hist_js = req.get(us_hist_covid).json()

covid_hist = pd.DataFrame(covid_hist_js)

covid_hist

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,totalTestResults,lastModified,total,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
0,20201020,56,8232367,109964576,8014.0,39230.0,438967.0,8178.0,22662.0,1889.0,...,126954814,2020-10-20T00:00:00Z,0,0,832,2150,757254,60664,909386,eab1ad654065bba93069314599177067b2545e3b
1,20201019,56,8171703,109207322,12138.0,37826.0,436817.0,7997.0,22475.0,1800.0,...,126045428,2020-10-19T00:00:00Z,0,0,456,1684,1025339,57148,1203168,c2555b3f55d9f948e0cff0db3790c2a3eaebfe20
2,20201018,56,8114555,108181983,12059.0,36437.0,435133.0,7292.0,22391.0,1762.0,...,124842260,2020-10-18T00:00:00Z,0,0,393,853,770454,48922,982434,dfcdff42c8ab6099a7f7cb67ba905b7210ed3a8a
3,20201017,56,8065633,107411529,12109.0,37359.0,434280.0,7384.0,22320.0,1791.0,...,123859826,2020-10-17T00:00:00Z,0,0,780,1535,954766,57943,1195546,0cc1293009cf57b75d14599ac356817f62371663
4,20201016,56,8007690,106456763,13091.0,37336.0,432745.0,7251.0,22202.0,1739.0,...,122664280,2020-10-16T00:00:00Z,0,0,877,1858,979899,68124,1143161,f89a417f68076f0145d099e9c12d4a900c5f67f9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,20200126,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2020-01-26T00:00:00Z,0,0,0,0,0,0,0,88a196879207bd090a61fb515beb301e84377977
269,20200125,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2020-01-25T00:00:00Z,0,0,0,0,0,0,0,43330e2f11d476ebcdd85e1a47551958e4c1e7ea
270,20200124,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2020-01-24T00:00:00Z,0,0,0,0,0,0,0,5f7a0cce04041596513629d55777a16f3aa65e8b
271,20200123,2,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2020-01-23T00:00:00Z,0,0,0,0,0,0,1,53f6a5e3985cebdead7ca275b0890ef975f0ccdd


In [71]:
## Change date field to Date
covid_hist['date'] = pd.to_datetime(covid_hist['date'], format='%Y%m%d')
covid_hist.head(20)

,date,states,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,totalTestResults,lastModified,total,posNeg,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease,hash
0,2020-10-20,56,8232367,109964576,8014.0,39230.0,438967.0,8178.0,22662.0,1889.0,...,126954814,2020-10-20T00:00:00Z,0,0,832,2150,757254,60664,909386,eab1ad654065bba93069314599177067b2545e3b
1,2020-10-19,56,8171703,109207322,12138.0,37826.0,436817.0,7997.0,22475.0,1800.0,...,126045428,2020-10-19T00:00:00Z,0,0,456,1684,1025339,57148,1203168,c2555b3f55d9f948e0cff0db3790c2a3eaebfe20
2,2020-10-18,56,8114555,108181983,12059.0,36437.0,435133.0,7292.0,22391.0,1762.0,...,124842260,2020-10-18T00:00:00Z,0,0,393,853,770454,48922,982434,dfcdff42c8ab6099a7f7cb67ba905b7210ed3a8a
3,2020-10-17,56,8065633,107411529,12109.0,37359.0,434280.0,7384.0,22320.0,1791.0,...,123859826,2020-10-17T00:00:00Z,0,0,780,1535,954766,57943,1195546,0cc1293009cf57b75d14599ac356817f62371663
4,2020-10-16,56,8007690,106456763,13091.0,37336.0,432745.0,7251.0,22202.0,1739.0,...,122664280,2020-10-16T00:00:00Z,0,0,877,1858,979899,68124,1143161,f89a417f68076f0145d099e9c12d4a900c5f67f9
5,2020-10-15,56,7939566,105476864,13001.0,37308.0,430887.0,7220.0,22051.0,1769.0,...,121521119,2020-10-15T00:00:00Z,0,0,951,2007,848075,63172,1019134,67bd987f3524bba5cfa9acb78d7d28c40528c3e7
6,2020-10-14,56,7876394,104628789,11100.0,37048.0,428880.0,7156.0,21889.0,1776.0,...,120501985,2020-10-14T00:00:00Z,0,0,811,2537,624821,56797,981061,d2b122eec06b897140b6821c08379f964657e001
7,2020-10-13,56,7819597,104003968,8008.0,36051.0,426343.0,7020.0,21735.0,1755.0,...,119520924,2020-10-13T00:00:00Z,0,0,690,2048,854225,48387,986044,b01088e611255e3c0fa9dc0148405e3a024e9c2e
8,2020-10-12,56,7771210,103149743,11329.0,35072.0,424295.0,6807.0,21625.0,1666.0,...,118534880,2020-10-12T00:00:00Z,0,0,287,1189,889434,43124,1016963,780501d348fbb458c1c39b13cc563ace52da82f5
9,2020-10-11,56,7728086,102260309,11261.0,34490.0,423106.0,6679.0,21553.0,1646.0,...,117517917,2020-10-11T00:00:00Z,0,0,466,1015,803858,46946,972008,96842f74faf7ca80657ba248de207cf3903bbdc7


In [72]:
keep_fields = ['date', 'positive', 'negative', 'death', 'deathIncrease', 'hospitalizedCurrently', 'onVentilatorCurrently', 'hospitalizedIncrease', 'positiveIncrease']

covid_hist = covid_hist[keep_fields]
covid_hist = covid_hist.rename(columns={'deathIncrease':'death_increase', 'hospitalizedCurrently':'hospitalized_currently', 'onVentilatorCurrently':'on_ventilator_currently', 'hospitalizedIncrease':'hospitalized_increase', 'positiveIncrease':'positive_increase'})
covid_hist


,date,positive,negative,death,death_increase,hospitalized_currently,on_ventilator_currently,hospitalized_increase,positive_increase
0,2020-10-20,8232367,109964576,212678.0,832,39230.0,1889.0,2150,60664
1,2020-10-19,8171703,109207322,211846.0,456,37826.0,1800.0,1684,57148
2,2020-10-18,8114555,108181983,211390.0,393,36437.0,1762.0,853,48922
3,2020-10-17,8065633,107411529,210997.0,780,37359.0,1791.0,1535,57943
4,2020-10-16,8007690,106456763,210217.0,877,37336.0,1739.0,1858,68124
...,...,...,...,...,...,...,...,...,...
268,2020-01-26,0,0,NaN,0,NaN,NaN,0,0
269,2020-01-25,0,0,NaN,0,NaN,NaN,0,0
270,2020-01-24,0,0,NaN,0,NaN,NaN,0,0
271,2020-01-23,0,0,NaN,0,NaN,NaN,0,0


In [73]:
covid_hist.to_sql(name='covid_hist', con=engine, if_exists='replace', index=False)


In [74]:
historical_df = pd.read_sql_query('select * from covid_hist', con=engine).head(15)
historical_df

,date,positive,negative,death,death_increase,hospitalized_currently,on_ventilator_currently,hospitalized_increase,positive_increase
0,2020-10-20,8232367,109964576,212678.0,832,39230.0,1889.0,2150,60664
1,2020-10-19,8171703,109207322,211846.0,456,37826.0,1800.0,1684,57148
2,2020-10-18,8114555,108181983,211390.0,393,36437.0,1762.0,853,48922
3,2020-10-17,8065633,107411529,210997.0,780,37359.0,1791.0,1535,57943
4,2020-10-16,8007690,106456763,210217.0,877,37336.0,1739.0,1858,68124
5,2020-10-15,7939566,105476864,209340.0,951,37308.0,1769.0,2007,63172
6,2020-10-14,7876394,104628789,208389.0,811,37048.0,1776.0,2537,56797
7,2020-10-13,7819597,104003968,207578.0,690,36051.0,1755.0,2048,48387
8,2020-10-12,7771210,103149743,206888.0,287,35072.0,1666.0,1189,43124
9,2020-10-11,7728086,102260309,206601.0,466,34490.0,1646.0,1015,46946


In [ ]:
#### Matt's Code Ends #### 